In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv("MovieMetadata.csv")

In [3]:
data_use = data.ix[:,['genres','plot_keywords','movie_title','actor_1_name',
                      'actor_2_name','actor_3_name','director_name','imdb_score']]

data_use['movie_title'] = [i.replace("\xa0","") for i in list(data_use['movie_title'])]

AttributeError: 'DataFrame' object has no attribute 'ix'

In [4]:
print(data_use.shape)
clean_data = data_use.dropna(axis = 0)
print(clean_data.shape)
clean_data = clean_data.drop_duplicates(['movie_title'])
clean_data = clean_data.reset_index(drop=True)
print(clean_data.shape)

(5043, 8)
(4781, 8)
(4659, 8)


In [11]:
people_list = []
for i in range(clean_data.shape[0]):
    name1 = clean_data.ix[i,'actor_1_name'].replace(" ","_")
    name2 = clean_data.ix[i,'actor_2_name'].replace(" ","_")
    name3 = clean_data.ix[i,'actor_3_name'].replace(" ","_")
    name4 = clean_data.ix[i,'director_name'].replace(" ","_")
    people_list.append("|".join([name1,name2,name3,name4]))
clean_data['people'] = people_list
print(clean_data['people'] )

0       CCH_Pounder|Joel_David_Moore|Wes_Studi|James_C...
1       Johnny_Depp|Orlando_Bloom|Jack_Davenport|Gore_...
2       Christoph_Waltz|Rory_Kinnear|Stephanie_Sigman|...
3       Tom_Hardy|Christian_Bale|Joseph_Gordon-Levitt|...
4       Daryl_Sabara|Samantha_Morton|Polly_Walker|Andr...
                              ...                        
4654    Carlos_Gallardo|Peter_Marquardt|Consuelo_Gómez...
4655    Richard_Jewell|John_Considine|Sara_Stepnicka|A...
4656    Kerry_Bishé|Caitlin_FitzGerald|Daniella_Pineda...
4657    Eric_Mabius|Daphne_Zuniga|Crystal_Lowe|Scott_S...
4658       John_August|Brian_Herzlinger|Jon_Gunn|Jon_Gunn
Name: people, Length: 4659, dtype: object


C:\Users\DHEVI\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DHEVI\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
C:\Users\DHEVI\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def token(text):
    return(text.split("|"))


cv_kw=CountVectorizer(max_features=100,tokenizer=token )
keywords = cv_kw.fit_transform(clean_data["plot_keywords"])
keywords_list = ["kw_" + i for i in cv_kw.get_feature_names()]

cv_ge=CountVectorizer(tokenizer=token )
genres = cv_ge.fit_transform(clean_data["genres"])
genres_list = ["genres_"+ i for i in cv_ge.get_feature_names()]

cv_pp=CountVectorizer(max_features=100,tokenizer=token )
people = cv_pp.fit_transform(clean_data["people"])
people_list = ["pp_"+ i for i in cv_pp.get_feature_names()]

cluster_data = np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list = keywords_list+genres_list+people_list

In [20]:
from sklearn.cluster import KMeans

mod = KMeans(n_clusters=100)
category = mod.fit_predict(cluster_data)
category_dataframe = pd.DataFrame({"category":category},index = clean_data['movie_title'])
print(category_dataframe)

                                          category
movie_title                                       
Avatar                                          15
Pirates of the Caribbean: At World's End        91
Spectre                                          7
The Dark Knight Rises                            7
John Carter                                     15
...                                            ...
El Mariachi                                     92
The Mongol King                                  8
Newlyweds                                       46
Signed Sealed Delivered                         46
My Date with Drew                               19

[4659 rows x 1 columns]


In [8]:
clean_data.ix[list(category_dataframe['category'] == 0),['genres','movie_title','people']]

C:\Users\DHEVI\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\DHEVI\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


,genres,movie_title,people
40,Adventure|Animation|Comedy|Family|Sport,Cars 2,Joe_Mantegna|Thomas_Kretschmann|Eddie_Izzard|J...
57,Adventure|Animation|Family|Sci-Fi,WALL·E,John_Ratzenberger|Fred_Willard|Jeff_Garlin|And...
66,Adventure|Animation|Comedy|Family,Up,John_Ratzenberger|Delroy_Lindo|Jess_Harnell|Pe...
67,Action|Adventure|Animation|Comedy|Family|Sci-Fi,Monsters vs. Aliens,Amy_Poehler|Rainn_Wilson|Stephen_Colbert|Rob_L...
88,Action|Adventure|Animation|Comedy|Drama|Family...,Big Hero 6,Damon_Wayans_Jr.|Daniel_Henney|Abraham_Benrubi...
...,...,...,...
3365,Adventure|Animation|Family,Snow Queen,Erin_Fitzgerald|Wendee_Lee|Ivan_Okhlobystin|Vl...
3540,Animation|Comedy|Family,Doug's 1st Movie,Frank_Welker|Constance_Shulman|Alice_Playten|M...
3621,Animation|Family,Henry & Me,Chazz_Palminteri|Cyndi_Lauper|Danny_Aiello|Bar...
4184,Adventure|Comedy|Family,Batman: The Movie,Burgess_Meredith|Cesar_Romero|Burt_Ward|Leslie...


In [9]:
def recommend(movie_name,recommend_number = 5):
    if movie_name in list(clean_data['movie_title']):
        movie_cluster = category_dataframe.ix[movie_name,'category']
        score = clean_data.ix[list(category_dataframe['category'] == movie_cluster),['imdb_score','movie_title']]
        sort_score = score.sort_values(['imdb_score'],ascending=[0])
        sort_score = sort_score[sort_score['movie_title'] != movie_name]
        recommend_number = min(sort_score.shape[0],recommend_number)
        recommend_movie = list(sort_score.iloc[range(recommend_number),1])
        print(recommend_movie)
    else:
        print("Can't find this movie!")

In [19]:
recommend('Despicable Me',10)

['A Charlie Brown Christmas', 'WALL·E', 'Up', 'Finding Nemo', 'The Incredibles', 'Big Hero 6', 'Fantastic Mr. Fox', 'Wreck-It Ralph', 'The Muppet Movie', 'Kung Fu Panda']


C:\Users\DHEVI\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DHEVI\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
